In [4]:
%matplotlib inline

# Imports
from Bio import SeqIO    
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt

In [7]:
# Functions
def getQualitites(filename, fmt=None):
    '''Get the qualities from the FASTQ'''
    if fmt is None:
        fmt = filename.split('.')[-1]
    
    L = max(len(read) for read in SeqIO.parse(filename, fmt))

    df = pd.DataFrame((read.letter_annotations['phred_quality'] + [np.nan] * (L - len(read))
                      for read in SeqIO.parse(filename, fmt)),
                      dtype=np.float64)
    return df

def readsToPanel (filename, fmt=None):
    '''Converts the sequences and quality scores from a sequence file to a Pandas panel.
    The items are 'sequence' and 'quality', the major axis is each read, the minor axis
    is the base or quality score.
    
    Parameters:
        filename = the name of the file
        fmt = the format of the file, defaults to the extension of the filename.
    '''
    if fmt is None:
        fmt = filename.split('.')[-1]
        
    size = max(len(read) for read in SeqIO.parse(filename, fmt))
            
    seqs = pd.DataFrame(((i) for i in read.seq) for read in SeqIO.parse(filename, fmt))
    
    quals = pd.DataFrame((read.letter_annotations['phred_quality'] + [np.nan] * (size - len(read))
                      for read in SeqIO.parse(filename, fmt)),
                      dtype=np.float64)
    
    data = pd.Panel ({'sequence' : seqs, 'quality' : quals})
    
    return data

def qualityByPosition(data):
    '''This function returns the mean of each base position in
    a read in a panel.

    Parameters
        data: The panel with the reads from the readsToPanel function.
    '''
    means = []
    for i in data.loc['quality']:
        means.append(np.mean(data.loc['quality'][i]))
    
    return means

def qualityByRead(data):
    '''This function returns the mean of each read in a panel.

    Parameters
        data: The panel with the reads from the readsToPanel function.
    '''
    means = []
    for i in data['quality'].transpose():
         means.append(np.mean(data['quality'].transpose()[i]))

    return means

def qualityByBase(data):
    '''This function prints the mean of each base in a dataframe.

    Parameters
        data: The panel with the reads from the readsToPanel function.
    '''
    a = []
    t = []
    c = []
    g = []
    
    for i, position in enumerate(data['sequence']):
        for j, base in enumerate(data['sequence'][i]):
            if base == 'A':
                a.append(data['quality'][i][j])
            elif base == 'T':
                t.append(data['quality'][i][j])
            elif base == 'C':
                c.append(data['quality'][i][j])
            elif base == 'G':
                g.append(data['quality'][i][j])

    print ("A: mean: ", np.mean(a), " standard deviation: ", np.std(a))
    print ("T: mean: ", np.mean(t), " standard deviation: ", np.std(t))
    print ("C: mean: ", np.mean(c), " standard deviation: ", np.std(c))
    print ("G: mean: ", np.mean(g), " standard deviation: ", np.std(g))
     
    return

def consensusSequence (data):
    '''Returns a string and a list of dictionaries. The string contains the consensus
    sequence selected by determining which base had the most 'votes' at a given position.
    Each dictionary contains the number of 'votes' for a base in a given position by
    tallying all the reads at that position.
    
    Parameters
        data: The panel with the reads from the readsToPanel function.
    '''
    rawConsensus = []
    string = ""
    for i in data.loc['sequence']:
        counts = { 'A' : 0, 'T' : 0, 'C' : 0, 'G' : 0 }
        for base in data.loc['sequence'][i]:
            if base == 'A':
                counts['A'] += 1
            elif base == 'T':
                counts['T'] += 1
            elif base == 'C':
                counts['C'] += 1
            elif base == 'G':  
                counts['G'] =+ 1
        # Consensus sequence looked strange so I wanted to see raw counts for each position.
        rawConsensus.append(counts)
        string += max(counts, key=counts.get)
    
    return string, rawConsensus
    

def qualityByFiveBases(data):
    '''NOT YET FUNCTIONAL
    
    This function identifies the sequence and average quality of the ten lowest 
    quality five base groups.

    Parameters
        data: The panel with the reads from the readsToPanel function.
    '''
    #start with quality by positions scores.
            #calculate mean of every five numbers
            #
    
    #dictionary with ten entries


def plotQualityAlongRead(df):
    '''Plot Phred quality along read'''
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    y = df.mean(axis=0)
    x = np.arange(len(y)) + 1
    dy = df.std(axis=0)
    ax.plot(x, y, lw=2, color='k')
    ax.fill_between(x, y-dy, y+dy, color='k', alpha=0.3)

    ax.set_xlabel('Position in read [bp]')
    ax.set_ylabel('Average phred quality at position')
    ax.grid(True)

    # Data for the subplot. Shape returns a tuple (an immutable list) of axes,
    # np.isnan returns a boolean array of nan values, sum adds each together on
    # the given axis. Result is a list with a value for reads per position.
    n_reads_pos = (df.shape[0] - np.isnan(df).sum(axis=0)) / 200.

    # Makes a bar graph, x coordinate of bar, height of bar, width, etc. bar edge width 
    ax.bar(x-.5, n_reads_pos, 1, facecolor='steelblue', edgecolor='blue', alpha=0.5, lw=1.5)

    # Adjust subplot paramters to fit graph.
    plt.tight_layout()
    

In [10]:
data = readsToPanel('../data/SRR2153267.fastq')

qualityByBase(data)

plt.hist(qualityByRead(data))
plotQualityAlongRead(data.loc['quality'])

consensusSequence(data)

FileNotFoundError: [Errno 2] No such file or directory: '../data/SRR2153267.fastq'